In [ ]:
!pip install -q openai tiktoken langchain langchain-community langchain-core magic_timer unstructured pydantic rouge_score==0.1.2 evaluate==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/

In [ ]:
import json
from pathlib import Path

import pickle
import pandas as pd
from magic_timer import MagicTimer

import openai
import tiktoken
from langchain_community.document_loaders import DirectoryLoader, TextLoader

from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')
MODEL_NAME = "gpt-4o-2024-05-13"

In [ ]:
path = "/content/drive/MyDrive/LLM - 22h + UFBA  + 🇩🇪/code_cronicas_rag/questions_answers_final.xlsx"

In [ ]:
df_questions_answer = pd.read_excel(path).filter(["question", "answer"])
df_questions_answer.head()

,question,answer
0,What effect does the global sialyltransferase ...,In vivo inhibition of sialylation with the glo...
1,What is required as a prerequisite for the dev...,Further studies of genetic markers are require...
2,What is the main focus of the model analyzed i...,The main focus of the model analyzed in the do...
3,What is used to monitor and adjust intravenous...,Anti-Xa level
4,What is a deletion hotspot in the context of t...,The poly(T)‐track in F8 intron 13 (c.2113+449_...


In [ ]:
def call_openai(model=None, prompt=None, history=None, params=None, sys_prompt=None):
    if model is None:
        return "Precisa passar o modelo openIA"
    else:
        default_kwargs = dict(model=model)

        if params:
            params = {**default_kwargs, **params}
        else:
            params = default_kwargs

        if history is None:
            history=[
                {'role':'system', 'content':sys_prompt},
                {'role':'user', 'content':prompt}
                ]
        else:
            history.append({'role':'user', 'content':prompt})


        response = openai.chat.completions.create(
            **params,
            messages=history,
            )

        usage = {
            "prompt_tokens":response.usage.prompt_tokens,
            "output_tokens":response.usage.completion_tokens,
            "total_tokens":response.usage.total_tokens,
        }

        output_model = response.choices[0].message.content
        history.append({"role":"assistant", "content":output_model})

        return usage, history, output_model

In [ ]:
create_system_prompt = """You always respond in American English.
Your role is as an expert in validating questions in the healthcare field
"""

evaluate_question_prompt = """You will receive a question about the healthcare field, using your expert knowledge, evaluate the quality of the question. Use the following score:

0: The question doesn't make sense to you or is poorly formulated.
1: The question makes sense but seems specific to some work or project.
2: The question makes sense and can be answered by a specialist with knowledge of the subject.
3: The question makes sense and can be answered by a specialist but is outside of your area of expertise.

Example:
Question: How many people with hemophilia are associated with adulthood?
Score: 0
Question: What types of hemophilia are presented in the study?
Score: 1
Question: What types of hemophilia are currently known?
Score: 2
Question: How many goals has Italy scored in World Cup matches?
Score: 3

The output must still be json with the characteristics of the following example:
Very calmly, make sure that the json does not contain errors.
Check and make sure there are no errors when loading this json in `json.loads` python code.

```
[
{{
"question_score": "Answer with 0, 1, 2 or 3 accordingly"
}},
]
```

Question: {question}
Score:

# The output must only be the file (json) without errors, it is very important that there are no errors when loaded by python's `json.loads`.
"""

create_openai_kwargs = {
    "response_format": {"type": "json_object"},
    "seed":341,
    "temperature":0,
}

In [ ]:
questions_score = []
for question in df_questions_answer["question"]:

  _, _, create_out = call_openai(
    model=MODEL_NAME,
    prompt=evaluate_question_prompt.format(question=question),
    history=None,
    params=create_openai_kwargs,
    sys_prompt = create_system_prompt
  )

  try:
      sample = json.loads(create_out)
      print("json ok")
  except:
      print(f"\tERROR")
  else:
      sample["question"] = question
      questions_score.append(sample)

json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok


In [ ]:
df_question_score = pd.DataFrame(questions_score)

In [ ]:
df_question_score["question_score"].value_counts()

question_score
2    84
1     8
0     1
Name: count, dtype: int64

---

In [ ]:
create_system_prompt = """You always respond in American English.
Your role is as a specialist in validating question and answer pairs in the healthcare field
"""

evaluate_question_answer_prompt = """You will receive a question and an answer about the healthcare field, using your specialized knowledge, evaluate the pair by answering the following three questions:

Is the answer correct? (Does it answer the question, yes or no)
Is the answer incomplete? (It answers the question, but as a specialist, you would expect other pertinent information to answer the question, yes or no)
Is the answer concise and direct? (Does it answer only what was asked, yes or no)


The output must still be json with the characteristics of the following example:
Very calmly, make sure that the json does not contain errors.
Check and make sure there are no errors when loading this json in `json.loads` python code.

```
[
{{
"Is the answer correct?": "Answer with yes or no here",
"Is the answer incomplete?": "Answer with yes or no here",
"Is the answer concise and direct?": "Answer with yes or no here"
}},
]
```

Question: {question}
Answer: {answer}
The output must only be the file (json) without errors, it is very important that there are no errors when loaded by python's `json.loads`.
"""

create_openai_kwargs = {
    "response_format": {"type": "json_object"},
    "seed":341,
    "temperature":0,
}


In [ ]:
pair_score = []
for idx, (question, answer) in df_questions_answer.iterrows():

  _, _, create_out = call_openai(
    model=MODEL_NAME,
    prompt=evaluate_question_answer_prompt.format(question=question, answer=answer),
    history=None,
    params=create_openai_kwargs,
    sys_prompt = create_system_prompt
  )

  try:
      sample = json.loads(create_out)
      print("json ok")
  except:
      print(f"\tERROR")
  else:
      sample["question"] = question
      pair_score.append(sample)

json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok
json ok


In [ ]:
df_pair_score = pd.DataFrame(pair_score)

In [ ]:
# juntando os doin e cruzando por question

In [ ]:
(
    df_pair_score.merge(df_question_score, on="question")
                 .rename(
                    columns = {
                        "Is the answer correct?": "A resposta responde à pergunta?",
                        "Is the answer incomplete?": "A resposta está Incompleta?",
                        "Is the answer concise and direct?": "A resposta e concisa e direta?",
                        "question_score":"model_question_score",
                        "question":"Questões"
                    }
                  )
                 .filter(["Questões",
                          "model_question_score",
                          "A resposta responde à pergunta?",
                          "A resposta está Incompleta?",
                          "A resposta e concisa e direta?"
                 ])
                 .to_csv("/content/drive/MyDrive/LLM - 22h + UFBA  + 🇩🇪/code_cronicas_rag/model_analysis.csv", index = False)
)